In [1]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
import shap

In [2]:
df = pd.read_csv("user_log_format1.csv")  # SHA256: a6bc8eabc40b61a562a5e45c91e9b822418d0d863576b28a9ccd549451e55a92
df1 = pd.read_csv("user_info_format1.csv")
df2 = pd.read_csv("train_format1.csv")
# df3 = pd.read_csv("test_format1.csv")
# df3 = df3[df3['user_id'].isin(unique_users)]
# df4 = pd.read_csv("train_format2.csv")
# df4 = df4[df4['user_id'].isin(unique_users)]
# df5 = pd.read_csv("test_format2.csv")
# df5 = df5[df5['user_id'].isin(unique_users)]

#print(df.shape)
#unique_users = df['user_id'].unique()
df = df[df['item_id'].isin(range(481, 640+1))]
#df2 = df2[df2['user_id'].isin(unique_users)]
df.sort_values('item_id')

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
41285064,84527,481,1331,318,4275.0,1005,0
23944557,417991,481,1331,318,4275.0,1008,0
23944670,417991,481,1331,318,4275.0,1012,0
24167652,288638,481,1331,318,4275.0,1009,0
6293884,20577,481,1331,318,4275.0,1110,0
...,...,...,...,...,...,...,...
33676891,43440,640,612,2246,394.0,925,0
9212183,363564,640,612,2246,394.0,918,0
51781467,232711,640,612,2246,394.0,1111,0
6134987,84505,640,612,2246,394.0,1111,0


In [3]:
expanded = pd.merge(pd.merge(df, df1, on='user_id'), df2, on='user_id')
# Remove rows with any nulls
expanded = expanded[expanded.notnull().all(axis=1)]
print(expanded.shape)
X = expanded.drop(columns=['label'])
y = expanded['label']
print(X.shape, y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
expanded

(5064, 11)
(5064, 10) (5064,)


,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type,age_range,gender,merchant_id,label
0,69247,568,683,270,397.0,1110,0,4.0,1.0,1999,0
1,69247,568,683,270,397.0,1110,0,4.0,1.0,2954,0
2,327584,489,420,3845,3156.0,708,0,4.0,0.0,2397,0
3,91170,640,612,2246,394.0,1111,0,5.0,0.0,1742,0
4,290029,495,464,4535,1720.0,1008,2,4.0,1.0,4585,0
...,...,...,...,...,...,...,...,...,...,...,...
5089,280765,496,267,2115,6973.0,1101,0,0.0,0.0,4765,0
5090,280765,496,267,2115,6973.0,1101,0,0.0,0.0,4765,0
5091,87449,504,1213,2264,3275.0,1015,0,0.0,1.0,1816,0
5092,87449,504,1213,2264,3275.0,1015,0,0.0,1.0,4518,0


In [4]:
# Let's see what works the best right out of the box
# We focus on RandomTreeClassifier and kNN for Shapley analysis
forest = RandomForestClassifier().fit(X_train, y_train)
print(forest.score(X_test, y_test))
knn = KNeighborsClassifier(n_neighbors=3).fit(X_train, y_train)
print(knn.score(X_test, y_test))

0.9585389930898321
0.947680157946693


In [5]:
# And let's test on which one does better when we do one-hot encoding on all the categorical columns.
expanded2 = pd.get_dummies(expanded, columns=['user_id', 'item_id', 'cat_id', 'seller_id', 'brand_id', 'action_type', 'age_range', 'merchant_id'])
print(expanded2.shape)
eX = expanded2.drop(columns=['label'])
ey = expanded2['label']
print(eX.shape, ey.shape)
eX_train, eX_test, ey_train, ey_test = train_test_split(eX, ey, test_size=0.2, random_state=1)
expanded2

(5064, 3531)
(5064, 3530) (5064,)


,time_stamp,gender,label,user_id_192,user_id_289,user_id_639,user_id_753,user_id_768,user_id_815,user_id_952,...,merchant_id_4955,merchant_id_4960,merchant_id_4966,merchant_id_4968,merchant_id_4975,merchant_id_4976,merchant_id_4988,merchant_id_4991,merchant_id_4992,merchant_id_4993
0,1110,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1110,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,708,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1111,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1008,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5089,1101,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5090,1101,0.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5091,1015,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5092,1015,1.0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
forest = RandomForestClassifier().fit(eX_train, ey_train)
print(forest.score(eX_test, ey_test))
knn = KNeighborsClassifier(n_neighbors=3).fit(eX_train, ey_train)
print(knn.score(eX_test, ey_test))

0.9684106614017769
0.930898321816387


In [7]:
pca = PCA(n_components=50)
user_ids = eX.filter(like='user_id_').columns
item_ids = eX.filter(like='item_id_').columns
cat_ids = eX.filter(like='cat_id_').columns
seller_ids = eX.filter(like='seller_id_').columns
brand_ids = eX.filter(like='brand_id_').columns
action_types = eX.filter(like='action_type_').columns
age_range = eX.filter(like='age_range_').columns
merchant_ids = eX.filter(like='merchant_id_').columns
print(user_ids)
onehots = ["gender"] + list(user_ids) + list(item_ids) + list(cat_ids) + list(seller_ids) + list(brand_ids) + list(action_types) +  list(age_range) + list(merchant_ids)

pca.fit(eX[onehots])
[f"{_:.4f}" for _ in pca.explained_variance_ratio_]
print(sum(pca.explained_variance_ratio_))

Index(['user_id_192', 'user_id_289', 'user_id_639', 'user_id_753',
       'user_id_768', 'user_id_815', 'user_id_952', 'user_id_1189',
       'user_id_1293', 'user_id_1408',
       ...
       'user_id_421504', 'user_id_421560', 'user_id_421752', 'user_id_421867',
       'user_id_422549', 'user_id_422987', 'user_id_423387', 'user_id_423555',
       'user_id_423772', 'user_id_424094'],
      dtype='object', length=2102)
0.6763719201742694


In [8]:
forest = RandomForestClassifier().fit(eX_train, ey_train)
print(forest.score(eX_test, ey_test))
knn = KNeighborsClassifier(n_neighbors=3).fit(eX_train, ey_train)
print(knn.score(eX_test, ey_test))

0.9674234945705824
0.930898321816387


In [9]:
pd.Series(forest.feature_importances_, index=eX.columns).sort_values().tail(20)

merchant_id_191     0.007690
user_id_267870      0.007890
merchant_id_1346    0.008001
user_id_316692      0.008068
user_id_114771      0.008107
user_id_112233      0.008533
user_id_80703       0.008569
age_range_4.0       0.008887
age_range_3.0       0.009226
merchant_id_2047    0.009519
user_id_26469       0.009820
merchant_id_4049    0.010180
merchant_id_586     0.010608
user_id_381946      0.010695
user_id_276113      0.010925
user_id_64342       0.012393
gender              0.013620
merchant_id_1116    0.015774
merchant_id_4943    0.016667
time_stamp          0.035602
dtype: float64

In [10]:
# Shapley time!
explainer = shap.TreeExplainer(forest, eX_train)
shap_values = explainer(eX_train.head(2))
print(type(explainer))

<class 'shap.explainers._tree.Tree'>


In [11]:
shap.summary_plot(shap_values[0], eX_train)

AssertionError: The shape of the shap_values matrix does not match the shape of the provided data matrix.